In [1]:
from transformers import BertTokenizer, BertTokenizerFast
import torch

from html import unescape
import re

from pprint import pprint
from tabulate import tabulate

from loadData import loadData
import const

In [2]:
save_directory = '.'

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["<E>", "</E>", "<URL>", "@USER"]})
tokenizer.add_tokens(["<E>", "</E>", "<URL>", "@USER"])
tokenizer.save_pretrained(save_directory)

('./vocab.txt', './special_tokens_map.json', './added_tokens.json')

In [7]:
train_dataloader_for_event = {}
subtask_list_for_event = {}

for event in const.EVENT_LIST:
    train_dataloader, dev_dataloader, test_dataloader, subtask_list = loadData(
        event, tokenizer, input_text_processing_func_list=[], train_ratio = 1)
    train_dataloader_for_event[event] = train_dataloader
    subtask_list_for_event[event] = subtask_list

Dataset Size Report: 26621 / 0 / 0 (train/dev/test)


Dataset Size Report: 13581 / 0 / 0 (train/dev/test)
Dataset Size Report: 15165 / 0 / 0 (train/dev/test)


Dataset Size Report: 15487 / 0 / 0 (train/dev/test)
Dataset Size Report: 12189 / 0 / 0 (train/dev/test)


# Gold Chunk Analysis

In this note, I am check if `gold_chunk_list` need any further processing, such as demoji, unescape and so on

In [ ]:
gold_chunk_set_for_subtask_for_event = {}
for event in const.EVENT_LIST:
    gold_chunk_set_for_subtask = {}
    for batch in train_dataloader_for_event[event]:
        for batch_data in batch['batch_data']:
            subtask_label_dict = batch_data[1]
            for subtask, (gold_chunk_list, gold_label) in subtask_label_dict.items():
            
                gold_chunk_set_for_subtask.setdefault(subtask, set())
                gold_chunk_set_for_subtask[subtask].update(gold_chunk_list)

    gold_chunk_set_for_subtask_for_event[event] = gold_chunk_set_for_subtask
    

## Stats of Unique Gold Chunk for each subtask at each event

In [11]:
for event in const.EVENT_LIST:
    print(f'Stats for event {event}')
    print(tabulate([(subtask, len(gold_chunk_set))
                    for subtask, gold_chunk_set in gold_chunk_set_for_subtask_for_event[event].items()],
                   headers=['Subtask', 'Unique Gold Chunk']))
    print('')


Stats for event positive
Subtask          Unique Gold Chunk
-------------  -------------------
age                             62
close_contact                  115
employer                       221
gender_male                    253
gender_female                  146
name                          1374
recent_travel                  114
relation                        79
when                            44
where                          424

Stats for event negative
Subtask          Unique Gold Chunk
-------------  -------------------
where                          107
age                             12
close_contact                   22
gender_male                    119
gender_female                   62
how_long                        22
name                           356
relation                        65
when                            21

Stats for event can_not_test
Subtask      Unique Gold Chunk
---------  -------------------
relation                   110
symptoms             

## Check chunks that change after preprocessing

Note this part may need to be done together with the input text to check if the results match each other.
(TODO Think about what's the influence of gold chunk. Maybe irrelevant once we have the correct gold label and process the input text all in the same form)